In [ ]:
!pip install dtreeviz

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from matplotlib import rcParams
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,VotingClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import BernoulliNB
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder,normalize,MinMaxScaler,Normalizer
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import confusion_matrix,roc_auc_score,roc_curve
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import plot_confusion_matrix
from sklearn.inspection import permutation_importance
import seaborn as sns
from imblearn.over_sampling import SMOTE
%matplotlib inline
import scipy as sp
import matplotlib.pyplot as plt

import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, roc_auc_score
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras import models
from sklearn.linear_model import LogisticRegression
import xgboost as xgb

from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

import pandas as pd
import seaborn as sns
from itertools import combinations
from IPython.display import display, HTML
from dtreeviz.trees import dtreeviz


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# **Defining two functions that are used to get a report of our datasets**

In [ ]:
def pretty_print(df):
    return display( HTML( df.to_html().replace("\\n","<br>") ) )
def tbl_report(tbl, cols=None, card=10):
    print("Table Shape", tbl.shape)
    dtypes = tbl.dtypes
    nulls = []
    uniques = []
    numuniques = []
    vcs = []
    for col in dtypes.index:
        n = tbl[col].isnull().sum()
        nulls.append(n)
        strdtcol = str(dtypes[col])
        #if strdtcol == 'object' or strdtcol[0:3] == 'int' or strdtcol[0:3] == 'int':
        #print(strdtcol)
        uniqs = tbl[col].unique()
        uniquenums = uniqs.shape[0]
        if uniquenums < card: # low cardinality
            valcounts = pd.value_counts(tbl[col], dropna=False)
            vc = "\n".join(["{}:{}".format(k,v) for k, v in valcounts.items()])
        else:
            vc='HC' # high cardinality
        uniques.append(uniqs)
        numuniques.append(uniquenums)
        vcs.append(vc)
    nullseries = pd.Series(nulls, index=dtypes.index)
    uniqueseries = pd.Series(uniques, index=dtypes.index)
    numuniqueseries = pd.Series(numuniques, index=dtypes.index)
    vcseries = pd.Series(vcs, index=dtypes.index)
    df = pd.concat([dtypes, nullseries, uniqueseries, numuniqueseries, vcseries], axis=1)
    df.columns = ['dtype', 'nulls', 'uniques', 'num_uniques', 'value_counts']
    if cols:
        return pretty_print(df[cols])
    return pretty_print(df)

# **Loading and Reading Datasets**

In [ ]:
#Loading train set and loading test set
train = pd.read_csv("/kaggle/input/higgs-boson/training.zip")
test = pd.read_csv("/kaggle/input/higgs-boson/test.zip")

#EventID is identifier - making it an index in both the sets
train.set_index('EventId',inplace = True)
test.set_index('EventId',inplace=True)

In [ ]:
#Looking at top 5 rows in train
train.head()

In [ ]:
#Looking at training set info 
tbl_report(train, cols=['dtype', 'nulls', 'num_uniques', 'value_counts'])

In [ ]:
#Looking at the numerical descriptions
train.describe()

In [ ]:
#Looking at top 5 rows in test
test.head()

In [ ]:
#Looking at test info
tbl_report(test, cols=['dtype', 'nulls', 'num_uniques', 'value_counts'])

In [ ]:
#Looking at statistical description of test
test.describe()

# **Exploratory Data Analysis**


In [ ]:
#Splitting into X and y
X = train.drop(['Weight','Label'],axis=1)
y = pd.factorize(train['Label'])[0]

## Let us look at the Class Ratio in our dataset.

In [ ]:
#Let's see the count of each class in our label
plt.figure(figsize=(10,8))
ax = sns.countplot(train['Label']);
for p in ax.patches:
        ax.annotate('{:d}'.format(p.get_height()), (p.get_x()+0.3, p.get_height()+5));

Looks like we do indeed have an imbalanced dataset. Considering this, accuracy would not be a good metric of performance. F1 score would be a better fit.

## Finding distribution of each Feature

In [ ]:
#Plotting Distribution of each feature
fig=plt.figure(figsize=(30,40))

for i in range(np.shape(train)[1]-1):
    ax = fig.add_subplot(8,4,i+1)
    ax = sns.distplot(train.iloc[:,i], color = 'dodgerblue')
    ax.set_title("Feature "+ train.columns[i] +" distribution")
    ax.legend()
fig.tight_layout();

## **Univariate Analysis**

### Finding the distribution of Data Per Class

In [ ]:
#Plotting Distribution of features per class
fig=plt.figure(figsize=(30,40))

for i in range(np.shape(train)[1]-2):
    ax = fig.add_subplot(6,5,i+1)
    ax = sns.distplot(train[train['Label'] == 's'].iloc[:,i],label="Class S", color = "blue")
    ax = sns.distplot(train[train['Label'] == 'b'].iloc[:,i],label="Class B", color = "grey")
    ax.set_title("Feature "+ train.columns[i] +" distribution per class")
    ax.legend()
fig.tight_layout();

## **Bivariate Analysis**

### Finding highly correlated Features - and printing the pairs of highly correlated features with threshold of 0.85

In [ ]:
train.corr().style.background_gradient(cmap='Blues')

### Printing out the highly correlated features along with their correlation

In [ ]:
#List to store the features with high correlation as tuples
feat=[]
#Setting a threshold of 0.9 of correlation
threshold=0.9
correlation=X.corr()
for i in X.columns:
    temp=correlation[i]
    #Finding the correlated features greater than the threshold
    corr_features=temp[(abs(temp)>threshold) & (temp.index!=i)].index.values
    #Adding the correlated features into a list keeping in mind that there is only one occurrence of the feature combination
    if(len(corr_features)!=0):
        for j in corr_features:
            features=(i,j)
        
            if(len(feat)==0):
                feat.append(features)
            else:
                count=len(feat)
                for x in feat:
                    if set(x) != set(features):
                        count-=1  
                    else:
                        break
                if(count==0):
                    feat.append(features)
                #[feat.append(features) for x in feat if not (set(x)==set(features))]

In [ ]:
print("The highly correlated features are given below")
for i in feat:
    corr=correlation[i[0]][i[1]]
    print('Features '+i[0]+' and '+i[1]+' are correlated with a correlation index of '+ str(np.round(corr,2)))

## **Getting the Data Ready**

Since we have an imbalanced dataset, we are using SMOTE for upsampling and downsampling our dataset.

## A) Dropping Correlated Features

In [ ]:
count = {}
for i, j in feat:
    if i in count.keys():
        count[i]+= 1
    else:
        count[i] = 1
    if j in count.keys():
        count[j]+= 1
    else:
        count[j] = 1
for k, v in count.items():
    if v > 2:
        X.drop(k, axis = 1, inplace = True)

In [ ]:
X.info()

## B) SMOTE Technique

In [ ]:
counter = Counter(pd.DataFrame(y))
print('Before', counter)

smt = SMOTE()

#oversampling using SMOTE
X_sm, y_sm = smt.fit_resample(X,y)

counter = Counter(y_sm)
print('After', counter)

In [ ]:
temp_df = pd.concat([X_sm,pd.DataFrame(y_sm,columns=['Label'])],axis=1)

f, axes = plt.subplots(figsize=(15, 10), dpi=100)
plt.subplot(121)
sns.despine()
sns.scatterplot(x=temp_df['PRI_met_sumet'], y=temp_df['DER_pt_ratio_lep_tau'], hue = temp_df['Label'], data=temp_df)
plt.title('Resampling with SMOTE', fontsize=14);

plt.subplot(122)
sns.despine()
sns.scatterplot(x=train['PRI_met_sumet'], y=train['DER_pt_ratio_lep_tau'], hue = train['Label'], data=train)
plt.title('No resampling', fontsize=14);

# **Creation of a Baseline Model**

In [ ]:
#Storing Categorical and Continuous Variables
cat_vars = ['PRI_jet_num']
cont_vars = np.array(train.drop(['PRI_jet_num','Weight','Label'],axis=1).columns)

## Utility Functions to make Model building and Cross-validation easier.

In [ ]:
def cv_optimize(clf, parameters, X, y, n_jobs=1, n_folds=5, score_func=None):
    if score_func:
        print("SCORE FUNC", score_func)
        gs = GridSearchCV(clf, param_grid=parameters, cv=n_folds, n_jobs=n_jobs, scoring=score_func)
    else:
        gs = GridSearchCV(clf, param_grid=parameters, n_jobs=n_jobs, cv=n_folds)
    gs.fit(X, y)
    print("BEST", gs.best_params_, gs.best_score_)
    best = gs.best_estimator_
    return best

def do_classify(clf, parameters, indf,y,score_func, n_folds=5, n_jobs=1):
    
    X = indf
    y = y
    
    Xtrain,Xtest,ytrain,ytest = train_test_split(X,y,train_size=0.8,random_state=214)
    
    clf = cv_optimize(clf, parameters, Xtrain, ytrain, n_jobs=n_jobs, n_folds=n_folds, score_func=score_func)
    
    clf = clf.fit(Xtrain, ytrain)
    
    training_accuracy = clf.score(Xtrain, ytrain)
    
    test_accuracy = clf.score(Xtest, ytest)
    print()
    print()
    print("############# Based On Standard Predict ################")
    print("Accuracy on training data: %0.2f" % (training_accuracy))
    print("Accuracy on test data:     %0.2f" % (test_accuracy))
    print()
    print(confusion_matrix(ytest, clf.predict(Xtest)))
    print("########################################################")
    print()
    
    plot_confusion_matrix(clf,Xtest,ytest,cmap="Blues")
    return clf, Xtrain, ytrain, Xtest,ytest

def p_importance(model, cols, fi, fistd = 0):
    return pd.DataFrame({'features':cols, 'importance':fi, 'importance_std': fistd}
                       ).sort_values('importance', ascending=False)

## Setting up Pipeline for Baseline Logistic Regression Model

In [ ]:
# Set Up Normalization
normalize = Normalizer()

# Set up One Hot Encoding of Target Variable
oh = OneHotEncoder()

# Continuous Variables Neet to be normalized
cont_pipe = Pipeline([('normalize', normalize)])

# Categorical variables need to be one hot encoded
cat_pipe = Pipeline([('onehot', oh)])

# Combine both into a transformer
transformers = [('cont', cont_pipe, cont_vars), ('cat', cat_pipe, cat_vars)]

# Apply transformer to relevant columns. Nothing will be done for the rest
ct = ColumnTransformer(transformers=transformers, remainder="passthrough")

# Create a pipeline so that we are not leaking data from validation to train in the individual folds
pipe_lr = Pipeline(steps=[('ct', ct), ('model', LogisticRegression(max_iter=10000, penalty='l2'))])

# In paramgrid we dont use C but use model__C corresponding to the name in the pipeline
paramgrid_lr = dict(model__C=[1000, 100, 0.1])

Now we train our model. Our do_classify takes care of subsetting the data and pickinging up the target variable.We score using the AUC on the validation sets.

In [ ]:
lr, X_train, y_train, X_test, y_test = do_classify(pipe_lr, paramgrid_lr, X_sm, y_sm, score_func='roc_auc')

## Setting up Pipeline for Random Forest Classifier

In [ ]:
# Create a pipeline so that we are not leaking data from validation to train in the individual folds
pipe_rf = Pipeline(steps=[('ct', ct), ('rf', RandomForestClassifier())])
paramgrid_rf = { 'rf__max_features': ['auto', 'sqrt'],
  'rf__max_depth': [5,10, 20, None], 
  'rf__min_samples_split': [2, 5, 10], 
  'rf__min_samples_leaf': [1, 2, 4],
  'rf__bootstrap': [True, False]}

rf, X_train, y_train, X_test, y_test = do_classify(pipe_rf, paramgrid_rf, X_sm, y_sm, score_func='roc_auc')

## Setting up Pipeline for Decision Tree Classifier

In [ ]:
# Create a pipeline so that we are not leaking data from validation to train in the individual folds
pipe_dt = Pipeline(steps=[('ct', ct), ('dt', DecisionTreeClassifier(random_state=142))])
paramgrid_dt = {'dt__max_depth':range(1,9),'dt__min_samples_leaf':range(3,5),'dt__criterion':['gini']}


clf, Xtrain, ytrain, Xtest,ytest = do_classify(pipe_dt, paramgrid_dt,X,y,'roc_auc',n_folds=5,n_jobs=-1)

In [ ]:
colors = [None, None,['red','blue'],]
dt_viz = dtreeviz(DecisionTreeClassifier(random_state=142), X,y,
               feature_names = X.columns,
               target_name = 'Label', class_names= ['S','B']
              ,orientation = 'TD',
               colors={'classes':colors},
               label_fontsize=12,
               ticks_fontsize=10,
               )

dt_viz.save("DecisionTreeClassifier.svg")

dt_viz

In [ ]:
dimp = permutation_importance(DecisionTreeClassifier(random_state=142),Xtest,ytest)
ddf = p_importance(DecisionTreeClassifier(random_state=142),list(X.columns),dimp['importances_mean'],dimp['importances_std']).iloc[:10]

fig,ax=plt.subplots(figsize=(17,10))
sns.barplot(data=ddf,x='features',y='importance',label='Decision_importances',ax=ax)
plt.xticks(rotation='45')
plt.title("Bar plot of Importances for Decision Tree Model");

## Trying out Boosting Models

In [ ]:
Xtrain,Xtest,ytrain,ytest = train_test_split(X , y ,train_size=0.8)

dtrain = xgb.DMatrix(Xtrain, label=ytrain)
dvalid = xgb.DMatrix(Xtest, label=ytest)

xgb_pars = {'min_child_weight': 100, 
            'eta': 0.04, 
            'colsample_bytree': 0.8, 
            'max_depth': 100,
            'subsample': 0.75, 
            'lambda': 2, 
            'nthread': -1, 
            'booster' : 
            'gbtree', 
            'silent': 1, 
            'gamma' : 0,
            'eval_metric': 'mae', 
            'objective': 'reg:linear'}    

boost = xgb.train(xgb_pars, dtrain, 500, maximize=False, verbose_eval=15) 

y_pred = clf3.predict(dvalid)
y_pred = [1 if y>0.5 else 0 for y in y_pred]

print(classification_report(ytest,y_pred))


## **Plotting and Comparing ROC Curves**

In [ ]:
def make_roc(name, clf, ytest, xtest, ax=None, labe=5,  proba=True, skip=0, initial = False):
    if not ax:
        ax=plt.gca()
    if proba:
        fpr, tpr, thresholds=roc_curve(ytest, clf.predict_proba(xtest)[:,1])
    else:
        fpr, tpr, thresholds=roc_curve(ytest, clf.decision_function(xtest))
    roc_auc = auc(fpr, tpr)
    if skip:
        l=fpr.shape[0]
        ax.plot(fpr[0:l:skip], tpr[0:l:skip], '.-', lw=2, alpha=0.4, label='ROC curve for %s (area = %0.2f)' % (name, roc_auc))
    else:
        ax.plot(fpr, tpr, '.-', lw=2, alpha=0.4, label='ROC curve for %s (area = %0.2f)' % (name, roc_auc))
    label_kwargs = {}
    label_kwargs['bbox'] = dict(
        boxstyle='round,pad=0.3', alpha=0.2,
    )
    for k in range(0, fpr.shape[0],labe):
        #from https://gist.github.com/podshumok/c1d1c9394335d86255b8
        threshold = str(np.round(thresholds[k], 2))
        ax.annotate(threshold, (fpr[k], tpr[k]), **label_kwargs)
    if initial:
        ax.plot([0, 1], [0, 1], 'k--')
        ax.set_xlim([0.0, 1.0])
        ax.set_ylim([0.0, 1.05])
        ax.set_xlabel('False Positive Rate')
        ax.set_ylabel('True Positive Rate')
        ax.set_title('ROC')
    ax.legend(loc="lower right")
    return ax

In [ ]:
make_roc('logistic', lr, ytest ,xtest, ax=ax, labe=10,  proba=True, skip=16300, initial = False)

In [ ]:
make_roc('randomForest', rf,ytest ,Xtest, ax=ax, labe=10,  proba=True, skip=16300, initial = False)

In [ ]:
make_roc('decisionTree', dt,ytest ,Xtest, ax=ax, labe=10,  proba=True, skip=16300, initial = False)

In [ ]:
make_roc('xgbboost', boost,ytest ,Xtest, ax=ax, labe=10,  proba=True, skip=16300, initial = False)

## **Self-supervised learning in Tabular Domain using VIME**

In [ ]:
#Setting Initial hyperparameters
# Experimental parameters
label_no = 1000  
model_sets = ['logit','xgboost','mlp']
  
# Hyper-parameters
p_m = 0.3
alpha = 0.01
K = 4
beta = 1.0
label_data_rate = 0.6

# Metric
metric = 'acc'
  
# Define output
results = np.zeros([len(model_sets)+2])  

In [ ]:
#Scaling data
scaler = StandardScaler()
X_sm = scaler.fit_transform(X_sm)

#Encoding Data
y_sm = np.where(y_sm == 'b',1,0)

### Splitting into unlabelled data and defining Utility functions

In [ ]:
#Splitting X and y into train and test 
x_train_, x_test_, y_train_, y_test_ = train_test_split(X_sm, y_sm, train_size=0.7, random_state=123)

#Spillting into unlabelled and labelled
idx = np.random.permutation(len(y_train))

# Label data : Unlabeled data = label_data_rate:(1-label_data_rate)
label_idx = idx[:int(len(idx)*label_data_rate)]
unlab_idx = idx[int(len(idx)*label_data_rate):]

# Unlabeled data
x_unlab = x_train.iloc[unlab_idx, :]

# Labeled data
x_train = x_train.iloc[label_idx, :]  
y_train = y_train[label_idx, :]

### Defining our Mask Generator
![image.png](attachment:image.png)


In [ ]:
#Mask Generator
def mask_generator (p_m, x):
    mask = np.random.binomial(1, p_m, x.shape)
    return mask

## Defining our Pre-text Generator

In [ ]:
#Pre-text Generator
def pretext_generator (m, x):  
    """Generate corrupted samples.

    Args:
    m: mask matrix
    x: feature matrix

    Returns:
    m_new: final mask matrix after corruption
    x_tilde: corrupted feature matrix
    """

    # Parameters
    no, dim = x.shape  
    
    # Randomly (and column-wise) shuffle data
    x_bar = np.zeros([no, dim])
    
    for i in range(dim):
        idx = np.random.permutation(no)
        x_bar[:, i] = x.iloc[idx, i]

    # Corrupt samples
    x_tilde = x * (1-m) + x_bar * m  
    
    # Define new mask matrix
    m_new = 1 * (x != x_tilde)

    return m_new, x_tilde

In [ ]:
#MLP
def mlp(x_train, y_train, x_test, parameters): 

    # Convert labels into proper format
    if len(y_train.shape) == 1:
        no = len(y_train)
        dim = len(np.unique(y_train))

        # Define output
        matrix = np.zeros([no,dim])

        # Convert vector to matrix
        for i in range(dim):
            idx = np.where(y_train == i)
            matrix[idx, i] = 1

        y_train = matrix

    # Divide training and validation sets (9:1)
    idx = np.random.permutation(len(x_train[:, 0]))
    train_idx = idx[:int(len(idx)*0.9)]
    valid_idx = idx[int(len(idx)*0.9):]

    # Validation set
    x_valid = x_train[valid_idx, :]
    y_valid = y_train[valid_idx, :]

    # Training set
    x_train = x_train[train_idx, :]
    y_train = y_train[train_idx, :]  

    # Define network parameters
    hidden_dim = parameters['hidden_dim']
    epochs_size = parameters['epochs']
    act_fn = parameters['activation']
    batch_size = parameters['batch_size']

    # Define basic parameters
    data_dim = len(x_train[0, :])
    label_dim = len(y_train[0, :])

    # Build model
    model = Sequential()
    model.add(Dense(hidden_dim, input_dim = data_dim, activation = act_fn))
    model.add(Dense(hidden_dim, activation = act_fn))
    model.add(Dense(hidden_dim, activation = act_fn))
    model.add(Dense(hidden_dim, activation = act_fn))
    model.add(Dense(label_dim, activation = 'sigmoid'))

    model.compile(loss = 'binary_crossentropy', optimizer='adam', 
                metrics = ['acc'])

    es = EarlyStopping(monitor='val_loss', mode = 'min', 
                     verbose = 1, restore_best_weights=True, patience=50)

    # Fit model on training dataset
    history = model.fit(x_train, y_train, validation_data = (x_valid, y_valid), 
            epochs = epochs_size, batch_size = batch_size, 
            verbose = 0, callbacks=[es])

    # Predict on x_test
    y_test_hat = model.predict(x_test)
    
    # evaluate model
    _, acc = model.evaluate(x_test, y_test, verbose=0)
    
    return y_test_hat, history, acc

In [ ]:
#Encoder

# Parameters
_, dim = x_unlab.shape
epochs = 30
batch_size = 128

# Build model  
inputs = Input(shape=(dim,))

# Encoder  
h = Dense(int(dim), activation='relu')(inputs)  

# Mask estimator
output_1 = Dense(dim, activation='sigmoid', name = 'mask')(h)  

# Feature estimator
output_2 = Dense(dim, activation='sigmoid', name = 'feature')(h)

model = Model(inputs = inputs, outputs = [output_1, output_2])

model.compile(optimizer='adam',
            loss={'mask': 'binary_crossentropy', 
                  'feature': 'mean_squared_error'},
            loss_weights={'mask':1, 'feature':alpha})

# Generate corrupted samples
m_unlab = mask_generator(p_m, x_unlab)
m_label, x_tilde = pretext_generator(m_unlab, x_unlab)

# Fit model on unlabeled data
encoder_history = model.fit(x_tilde, {'mask': m_label, 'feature': x_unlab}, 
        epochs = epochs, batch_size= batch_size)

# Extract encoder part
layer_name = model.layers[1].name
layer_output = model.get_layer(layer_name).output
encoder = models.Model(inputs=model.input, outputs=layer_output)

In [ ]:
plt.figure(figsize=(18,7))
# plot loss
plt.title('Mask Loss')
plt.plot(encoder_history.history['mask_loss'], color='orange', label='Mask Loss')
plt.plot(encoder_history.history['feature_loss'], color='red', label='Feature Loss')
plt.legend();

In [ ]:
mlp_parameters = dict()
mlp_parameters['hidden_dim'] = 256
mlp_parameters['epochs'] = 1000
mlp_parameters['activation'] = 'sigmoid'
mlp_parameters['batch_size'] = 128

x_train_hat = encoder.predict(x_train)
x_test_hat = encoder.predict(x_test)
      
y_test_hat, MLP_history, acc = mlp(x_train_hat, y_train, x_test_hat, mlp_parameters)

print('VIME-Self Accuracy: ' + str(np.round(acc,2)))

In [ ]:
plt.figure(figsize=(18,7))
# plot loss
plt.subplot(121)
plt.title('Cross Entropy Loss')
plt.plot(MLP_history.history['loss'], color='blue', label='train')
plt.plot(MLP_history.history['val_loss'], color='orange', label='test')
plt.legend()

# plot accuracy
plt.subplot(122)
plt.title('Classification Accuracy')
plt.plot(MLP_history.history['acc'], color='blue', label='train')
plt.plot(MLP_history.history['val_acc'], color='orange', label='test')
plt.legend();